<a href="https://colab.research.google.com/github/Jaime44/KUNVENO/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

In [1]:
import sys
import os


# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/PRÁCITCAS EXT/KUNVENO/KUNVENO_DIGITAL/'


pat_dir_checkpoints = path_absolute+'checkpoints'

El código se está ejecutando en Google Colab.
Mounted at /content/drive
Contenido de la carpeta en Google Drive:
['Git_commnads.txt', '.git', 'README.md', 'checkpoints', 'icefall', '.gitmodules', '.gitignore', 'icefall_yes_no_dataset_recipe_with_CPU.ipynb', 'pruned_transducer_statelessx_with_GPU.ipynb']


In [2]:
!ls -la

total 88
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   439 Oct 25 18:36 .gitignore
-rw------- 1 root root    79 Oct 16 19:29 .gitmodules
drwx------ 2 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 55078 Oct 26 19:02 icefall_yes_no_dataset_recipe_with_CPU.ipynb
-rw------- 1 root root 18018 Oct 29 16:53 pruned_transducer_statelessx_with_GPU.ipynb
-rw------- 1 root root    17 Oct 11 16:57 README.md


In [3]:
!pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


### Install PyTorch and torchaudio

In [4]:
import torch
print(torch.__version__)

2.1.0+cu118


Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [5]:
! pip install torchaudio==2.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [6]:
# ! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html
! pip install k2==1.24.3.dev20230718+cuda11.7.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 MB 7.0 MB/s eta 0:00:00


Check that k2 was installed successfully:

In [7]:
! python3 -m k2.version

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/k2/__init__.py", line 23, in <module>
    from _k2 import DeterminizeWeightPushingType
ImportError: libcuda.so.1: cannot open shared object file: No such file or directory


### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [8]:
! pip install lhotse
! pip install git+https://github.com/lhotse-speech/lhotse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.9/731.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 9.2 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=e2e7bbda7c9fdf05290040c0d39c50550cd8e208847fa5c4f4c3804a69cf8122
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-ycr_58id
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-ycr_58id
  Resolved https://github.com/lhotse-speech/lhotse to commit 44bc1ab5ecbcdfa134d5132100ef77d8bfb554b7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lhotse: filename=lhotse-1.17.0.dev0+git.44bc1ab.clean-py3-none-any.whl size=738420 sha256=f67f69070b5ac36a9db37e77f54501af873029871aa6f589047492d6bb4cba23
  Stored in directory: /tmp/pip-ephem-wheel-cache-yuqzwf5k/wheels/df/b0/ff/cce0f16868fcdbee2088f3acf9f249dc90117d5f5dd9b6f69d
Successfully built lhotse
  Attempting uninstall: lhotse
    Found existing installation: lhotse 1.17.0
    Uninstalling lhotse-1.17.0:
      Successfully uninstalled lhotse-1.17.0


### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [9]:
! pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


Now install dependencies of `icefall`:

In [10]:
!pip install typing-extensions==3.10.0.2


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.22 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
arviz 0.15.1 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
chex 0.1.7 requires typing-extensions>=4.2.0; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
flax 0.7.4 requires typing-extensions>=4.2, but you have typing-extensions 3.10.0.2 which is incompatible.
ibis-framework 6.2.0 requires typing-extensions<5,>=4.3.0, but you have typing-extensions 3.10.0.2 which is incompatible.
librosa 0.10.1 requires typing-extensio

In [11]:
! cd icefall && \
  pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4

## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [12]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Successfully uninstalled tensorflow-2.14.0


In [13]:
!sudo chmod -R 755 icefall/egs/librispeech/ASR/

In [14]:
!rm -r icefall/egs/librispeech/ASR/shared && \
  sudo ln -s ../../../icefall/shared/ icefall/egs/librispeech/ASR/shared


In [15]:
!rm -r icefall/icefall/transformer_lm/scaling.py && \
ls -la icefall/icefall/transformer_lm/


total 69
-rw------- 1 root root 22349 Oct 12 19:52 attention.py
-rw------- 1 root root  5497 Oct 12 19:52 compute_perplexity.py
-rw------- 1 root root    20 Oct 12 19:52 dataset.py
-rw------- 1 root root 11234 Oct 12 19:52 encoder.py
-rw------- 1 root root  4925 Oct 12 19:52 export.py
-rw------- 1 root root     0 Oct 12 19:52 __init__.py
-rw------- 1 root root  3396 Oct 12 19:52 model.py
drwx------ 2 root root  4096 Oct 12 19:56 __pycache__
-rw------- 1 root root 17243 Oct 12 19:52 train.py


In [16]:
!sudo ln -s ../../egs/librispeech/ASR/pruned_transducer_stateless2/scaling.py icefall/icefall/transformer_lm/scaling.py

In [17]:
# !sudo ln -s ../../../librispeech/ASR/local/prepare_lang_fst.py icefall/egs/yesno/ASR/local/prepare_lang_fst.py && \
# sudo chmod 755 icefall/egs/yesno/ASR/local/prepare_lang_fst.py

In [18]:
!apt-get update && apt-get install dos2unix

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [44.8 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,131 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,009 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,400 kB]
Get:13 http://archive.ubuntu.co

In [19]:
# !dos2unix icefall/egs/yesno/ASR/prepare.sh
# !dos2unix icefall/icefall/shared/parse_options.sh

In [20]:
# !find icefall -type f -print0 | xargs -0 dos2unix

In [21]:
!sudo chmod 755 icefall/egs/librispeech/ASR/prepare.sh

In [22]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd icefall/egs/librispeech/ASR && \
#   ./prepare.sh
#   # rm -rf data && \

In [23]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd icefall/egs/librispeech/ASR && \
#   ./prepare.sh --stage 1

In [24]:
!sudo chmod 755 icefall/egs/librispeech/ASR/pruned_transducer_stateless4/train.py

In [25]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [26]:
!rm -r icefall/egs/aishell/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
ln -s ../tdnn_lstm_ctc/asr_datamodule.py icefall/egs/aishell/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
ln -s ../transducer/asr_datamodule.py icefall/egs/librispeech/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/asr_datamodule.py && \
ln -s ../pruned_transducer_stateless2/asr_datamodule.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/asr_datamodule.py && \
rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py && \
ln -s ../pruned_transducer_stateless2/optim.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py

In [27]:
!ls -la icefall/egs/aishell/ASR/pruned_transducer_stateless2

total 69
lrw------- 1 root root    34 Oct 29 17:03 asr_datamodule.py -> ../tdnn_lstm_ctc/asr_datamodule.py
-rw------- 1 root root    68 Oct 12 19:35 beam_search.py
-rw------- 1 root root    66 Oct 12 19:35 conformer.py
-rw------- 1 root root 17117 Oct 12 19:35 decode.py
-rw------- 1 root root    64 Oct 12 19:35 decoder.py
-rw------- 1 root root    74 Oct 12 19:35 encoder_interface.py
-rw------- 1 root root  6236 Oct 12 19:35 export.py
-rw------- 1 root root    63 Oct 12 19:35 joiner.py
-rw------- 1 root root    62 Oct 12 19:35 model.py
-rw------- 1 root root    62 Oct 12 19:36 optim.py
-rw------- 1 root root  9284 Oct 12 19:35 pretrained.py
-rw------- 1 root root    64 Oct 12 19:35 scaling.py
-rw------- 1 root root 32062 Oct 12 19:36 train.py


In [28]:
!ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless2

total 332
lrwx------ 1 root root     31 Oct 29 17:03 asr_datamodule.py -> ../transducer/asr_datamodule.py
-rwx------ 1 root root 100529 Oct 12 19:40 beam_search.py
-rwx------ 1 root root  64089 Oct 12 19:40 conformer.py
-rwx------ 1 root root  25947 Oct 12 19:41 decode.py
-rwx------ 1 root root   4425 Oct 12 19:40 decoder.py
-rwx------ 1 root root     47 Oct 12 19:40 decode_stream.py
-rwx------ 1 root root     44 Oct 12 19:40 encoder_interface.py
-rwx------ 1 root root   6780 Oct 12 19:40 export.py
-rwx------ 1 root root      0 Oct 12 19:40 __init__.py
-rwx------ 1 root root   2181 Oct 12 19:41 joiner.py
-rwx------ 1 root root   7445 Oct 12 19:41 model.py
-rwx------ 1 root root  11877 Oct 12 19:41 optim.py
-rwx------ 1 root root  11429 Oct 12 19:41 pretrained.py
-rwx------ 1 root root  36364 Oct 12 19:41 scaling.py
-rwx------ 1 root root   9492 Oct 12 19:41 streaming_beam_search.py
-rwx------ 1 root root  18386 Oct 12 19:41 streaming_decode.py
-rwx------ 1 root root     44 Oct 12 19:41

In [29]:
!ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless4

total 114
lrwx------ 1 root root    49 Oct 29 17:03 asr_datamodule.py -> ../pruned_transducer_stateless2/asr_datamodule.py
-rwx------ 1 root root    46 Oct 12 19:41 beam_search.py
-rwx------ 1 root root    44 Oct 12 19:41 conformer.py
-rwx------ 1 root root 32416 Oct 12 19:41 decode.py
-rwx------ 1 root root    42 Oct 12 19:41 decoder.py
-rwx------ 1 root root    47 Oct 12 19:41 decode_stream.py
-rwx------ 1 root root    52 Oct 12 19:41 encoder_interface.py
-rwx------ 1 root root  9492 Oct 12 19:41 export.py
-rwx------ 1 root root    43 Oct 12 19:41 __init__.py
-rwx------ 1 root root    41 Oct 12 19:41 joiner.py
-rwx------ 1 root root    40 Oct 12 19:41 lstmp.py
-rwx------ 1 root root    40 Oct 12 19:41 model.py
lrwx------ 1 root root    40 Oct 29 17:03 optim.py -> ../pruned_transducer_stateless2/optim.py
-rwx------ 1 root root  4030 Oct 12 19:41 profile.py
drwx------ 2 root root  4096 Oct 29 13:24 __pycache__
-rwx------ 1 root root    52 Oct 12 19:41 scaling_converter.py
-rwx------ 1 

In [30]:
!export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
  ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless2 && \
  ls -la icefall/egs/aishell/ASR/pruned_transducer_stateless2 && \
  cd icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
  ./train.py --help

total 114
lrwx------ 1 root root    49 Oct 29 17:03 asr_datamodule.py -> ../pruned_transducer_stateless2/asr_datamodule.py
-rwx------ 1 root root    46 Oct 12 19:41 beam_search.py
-rwx------ 1 root root    44 Oct 12 19:41 conformer.py
-rwx------ 1 root root 32416 Oct 12 19:41 decode.py
-rwx------ 1 root root    42 Oct 12 19:41 decoder.py
-rwx------ 1 root root    47 Oct 12 19:41 decode_stream.py
-rwx------ 1 root root    52 Oct 12 19:41 encoder_interface.py
-rwx------ 1 root root  9492 Oct 12 19:41 export.py
-rwx------ 1 root root    43 Oct 12 19:41 __init__.py
-rwx------ 1 root root    41 Oct 12 19:41 joiner.py
-rwx------ 1 root root    40 Oct 12 19:41 lstmp.py
-rwx------ 1 root root    40 Oct 12 19:41 model.py
lrwx------ 1 root root    40 Oct 29 17:03 optim.py -> ../pruned_transducer_stateless2/optim.py
-rwx------ 1 root root  4030 Oct 12 19:41 profile.py
drwx------ 2 root root  4096 Oct 29 13:24 __pycache__
-rwx------ 1 root root    52 Oct 12 19:41 scaling_converter.py
-rwx------ 1 

In [31]:
! nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


------------------------------------------------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------------------------------------------

## Training

In [32]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
# cd icefall/egs/yesno/ASR && \
# ./tdnn/train.py

In [33]:
# !ls -la


In [34]:
# !cd icefall/egs/yesno/ASR/tdnn/exp/tensorboard && \
# tensorboard dev upload --logdir . --description "TDNN training for yesno with icefall"

## Decoding

In [35]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/decode.py

### Show the decoding result

In [36]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cat tdnn/exp/recogs-test_set.txt

### Show the detailed WER

In [37]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cat tdnn/exp/errs-test_set.txt

# Pre-trained model

### Download the pre-trained model

In [38]:
# # ! cd /content/icefall/egs/yesno/ASR && \
# #   mkdir tmp && \
# #   cd tmp && \
# #   git lfs install && \
# #   git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cd tmp &&

In [39]:
# #! sudo apt-get install git-lfs

In [40]:
# ! sudo apt-get install tree

In [41]:
# # ! cd /content/icefall/egs/yesno/ASR && \
# #   mkdir -p tmp && \
# #   tree tmp

# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   tree tmp

In [42]:
# ! sudo apt-get install sox

In [43]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [44]:
# ! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

## Inference with a pre-trained model


### View help information

In [45]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py --help

### Decode a single sound file

In [46]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

### Decode multiple sound files

In [47]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav